In [ ]:
ENVIRONMENT = None # must set to "local" or "remote" before running

In [ ]:
import os
import shutil
from datetime import datetime, timedelta
import urllib
import zipfile
import matplotlib.pyplot as plt
import requests
import pandas as pd

#from configs import EXT_DATA_DIR, FRD_USER_ID, FRD_DATA_DIR
# configs

# ask for environment via input
if ENVIRONMENT is None:
    ENVIRONMENT = input("Please enter environment (local/remote): ")
    assert ENVIRONMENT in ["local", "remote"], "Invalid environment"

with open(FRD_USER_ID_FP, "r") as file:
    FRD_USER_ID = file.read().strip()
if ENVIRONMENT == "local":
    if os.name == "nt":
        PROJECT_DIR = "C:\\Users\\regin\\Dropbox\\ibis"
        FRD_DATA_DIR = "E:\\frd-historical\\data\\"
        FRD_USER_ID_FP = "C:\\Users\\regin\\Dropbox\\API_KEYS\\FRD-USER-ID"
    else:
        PROJECT_DIR = "/home/reggie/Dropbox/ibis/"
        FRD_DATA_DIR = "/media/reggie/reg_ext/frd-historical/data/"
        FRD_USER_ID_FP = "/home/reggie/Dropbox/API_KEYS/FRD-USER-ID"
elif ENVIRONMENT == "remote":
    PROJECT_DIR = "/home/ubuntu/ibis/"
    FRD_DATA_DIR = "/home/ubuntu/ibis/frd-historical/data/"
    FRD_USER_ID_FP = "/home/ubuntu/FRD-USER-ID"

DATA_DIR = os.path.join(PROJECT_DIR, "data")
#FRD_DATA_DIR = os.path.join(DATA_DIR, "frd-historical")

# Trading calendar, holidays
calendar = {
    "july4":["2023-07-03"],
    "911":["2023-09-11"],
    "thanksgiving":["2023-11-24"],
    "xmas":["2023-12-25"],
    "nyd":["2023-01-01"],
}

print(FRD_DATA_DIR, os.listdir(FRD_DATA_DIR))
print(DATA_DIR, os.listdir(DATA_DIR))

# Ticker Listing
* Get available stock (for now) ticker symbols from FRD.
* Store the sheet in `save_fp = os.path.join(FRD_DATA_DIR, "stock", "ticker_listing.csv")`.
* Add a column if it seems that the stock was delisted.


In [ ]:
"""
Ticker Listing
This function returns the full listing of tickers as well as start and end dates for a specified instrument type.
The data is returned in csv format so it can be copied into a text file and opened from a spreadsheet application if required.

Url EndPoint : https://firstratedata.com/api/ticker_listing

Data Format : {ticker},{name},{startDate},{endDate}

Parameters : The below parameters are used with the Url Endpoint to use the Last Update function:

Parameter : type
Accepted Values : stock , etf(only stocks or ETFs can currently be requested)

Description : Specifies the type of instrument that is being requested.

Example :
https://firstratedata.com/api/ticker_listing?type=stock&userid=c85lvfWKHUivhqC3fDJlBw

Parameter : html
Accepted Values : true, false (false is the default value)

Description : Specifies the is the returned data is in HTML format. Set this value to true to view the data in a web browser.

Example :
https://firstratedata.com/api/ticker_listing?type=stock&userid=c85lvfWKHUivhqC3fDJlBw&html=true
"""

def ticker_listing(type, save_fp="", html=False):
    """
    Parameters
    ----------
    type : string
        Accepted Values : stock , etf(only stocks or ETFs can currently be requested)
        Description : Specifies the type of instrument that is being requested.

    html : boolean
        Accepted Values : true, false (false is the default value)
        Description : Specifies the is the returned data is in HTML format. Set this value to true to view the data in a web browser.
    """
    url = "https://firstratedata.com/api/ticker_listing?type={}&userid={}&html={}".format(type, FRD_USER_ID, html)

    # download
    try:
        urllib.request.urlretrieve(url, save_fp)
        print("Downloaded ticker listing to {}".format(save_fp))

        ticker_listing_df = pd.read_csv(save_fp)
        ticker_listing_df.columns = ["ticker", "name", "first_date", "last_date"]

        # Apply the function to the 'last_date' column to create the 'delisted' flag
        # TODO: Improve because this has edge cases with zero volume trading days
        ticker_listing_df['delisted'] = ticker_listing_df['last_date'].apply(is_delisted)
        ticker_listing_df.to_csv(save_fp, index=False)
        return 0
    except:
        print("Error: Could not download ticker listing.")

    # save ticker listing

    return

def is_delisted(last_date_str: str) -> int:
    """
    Function to check if the last_date is before the most recent weekday.
    Doesn't necessarily require a str input, could be a datetime object.
    Parameters
    ----------
    last_date_str : string
        Last date of the stock data    
    """
    last_date = datetime.strptime(last_date_str, '%Y-%m-%d').date()
    today = datetime.now().date() - timedelta(days=7)
    # Adjust today to the most recent weekday if it's a weekend
    if today.weekday() > 4:  # 0 is Monday, 6 is Sunday
        today -= timedelta(days=today.weekday() - 4)
    return int(last_date < today)


# Querying the First Rate Data Historical Stock Database API

## Process
<ol> 
<li>Download to ext </li>
<li>Compress on ext</li>
<li>move to local HD</li>
</ol>


## Download list
* full series for 1min, 5min, 30min, 1hour, and 1day for stocks, etfs, index, crypto, fx, futures, and options
* include (separately) splitdiv adjusted, split adjusted, and unadjusted
* Get dividends and splits 


## Directory Structure
* foo

In [ ]:
save_fp = os.path.join(FRD_DATA_DIR, "stock", "ticker_listing.csv")
#ticker_listing("stock", save_fp=save_fp,)
ticker_listing_df = pd.read_csv(save_fp)
print(ticker_listing_df.shape)
ticker_listing_df.head()

In [ ]:
ticker_listing_df['ticker_first_letter'] = ticker_listing_df['ticker'].str[0]
ticker_listing_df.sort_values('ticker_first_letter', inplace=True)
ticker_first_letter_counts = ticker_listing_df['ticker_first_letter'].value_counts().reset_index()
ticker_first_letter_counts.sort_values("ticker_first_letter", inplace=True)
ticker_first_letter_counts
plt.bar(ticker_first_letter_counts['ticker_first_letter'], ticker_first_letter_counts['count'])
plt.show()

# Historical Data
See https://firstratedata.com/about/api-docs?userid=c85lvfWKHUivhqC3fDJlBw#datarequest

## Download

### Download SPY S&P 500 Index data

In [ ]:
# Example downloading SPY
ticker = "SPY"
ticker_first_letter = ticker[0]
type = "etf"
period = "full"
timeframe = "1min"
adjustment = "adj_split"
base_url = 'https://firstratedata.com/api/data_file'

frd_api_params = {
    'type': type,
    'period': period,
    'ticker_range': ticker_first_letter,
    'timeframe': timeframe,
    'adjustment': adjustment,
    'userid': FRD_USER_ID
}
download_filename = "temp/tmp.zip"
response = requests.get(base_url, params=frd_api_params)
with open(download_filename, 'wb') as f:
    f.write(response.content)

extract_zip(download_filename, f"temp/etfs/{ticker_first_letter}/")


In [ ]:
adjustment = adjustment.replace("_", "")
spy_filepath = f"temp/etfs/{ticker_first_letter}/{ticker}_{period}_{timeframe}_{adjustment}.txt"
column_names = ["date", "open", "high", "low", "close", "volume"]
print(spy_filepath)
spy_df = pd.read_csv(spy_filepath, header=None, names=column_names, parse_dates=["date"], index_col="date")
print(spy_df.shape)
spy_df

### Stocks

In [ ]:
# check if windows or linux
if os.name == 'nt':
    print('Windows')
    ZIP_FILES_DIR = os.path.join(EXT_DATA_DIR, "stock", timeframe, "zips")
else:
    print('Linux')
    ZIP_FILES_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/zips/"
    
def make_download_path(**params):
    """
    This is for First Rate Data downloads (zip files.)
    Make a download path for the given parameters. 
    If the path does not exist, create it.
    """
    adjustment = params['adjustment']
    timeframe = params['timeframe']
    ticker_first_letter = params['ticker_first_letter']    
    
    fp = os.path.join(
        ZIP_FILES_DIR,
        adjustment,
        timeframe,
    )
    if ticker_first_letter:
        fp = os.path.join(fp, ticker_first_letter)
    if not os.path.exists(fp):
        print("making directory {}".format(fp))
        os.makedirs(fp)
    return fp

def download_stock_data(**params):
    """
    Download stock data from FirstRateData API.

    Parameters
    ----------
    ticker_first_letter : string
        First letter of the ticker symbol to download data for.
    timeframe : string
        Timeframe to download data for. Accepted values are: 1min, 5min, 30min, 1hour, 1day
    adjustment : string
        Adjustment type to download data for. Accepted values are: adj_split, adj_splitdiv, unadjusted

    Returns
    -------
    None

    Examples
    --------
    >>> download_stock_data('A', '1min', 'adj_split')
    Downloaded: data/stock/A/adj_split/1min/A_adj_split_1min.zip
    """
    # Unpack parameters
    asset_type = params['asset_type']
    period = params['period']
    ticker_first_letter = params['ticker_first_letter']
    timeframe = params['timeframe']
    adjustment = params['adjustment']
    overwrite = params['overwrite']

    ZIP_FILES_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/zips/"
    if not os.path.exists(ZIP_FILES_DIR):
        os.makedirs(ZIP_FILES_DIR)
        
    # Define API URL
    base_url = 'https://firstratedata.com/api/data_file'

    frd_api_params = {
        'type': asset_type,
        'period': period,
        'ticker_range': ticker_first_letter,
        'timeframe': timeframe,
        'adjustment': adjustment,
        'userid': FRD_USER_ID
    }
    print(ticker_first_letter)
    filename = f"{ticker_first_letter}_{adjustment}_{timeframe}.zip"
    zips_filename = os.path.join(ZIP_FILES_DIR, filename)
    print(zips_filename)

    #download_path = make_download_path(**params)
    download_filename = os.path.join(ZIP_FILES_DIR, filename)
    if os.path.exists(zips_filename) and overwrite is False:
        print(f"Already downloaded {ticker_first_letter} {period} data for {timeframe} and {adjustment} to {download_filename}")
    else:
        if not os.path.exists(zips_filename) or overwrite is True:
            print(f"Downloading {ticker_first_letter} {period} data for {timeframe} and {adjustment} to {download_filename}")
            response = requests.get(base_url, params=frd_api_params)
            if response.status_code == 200:
                with open(download_filename, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded: {download_filename}")
            else:
                print(f"Failed to download {ticker_first_letter} data for {timeframe} and {adjustment}. Status: {response.status_code}")
    return

In [ ]:

params = {
    "asset_type": "stock",
    "ticker_first_letter": "Z",
    #"timeframe": "5min",
    "timeframe": "1min",
    "adjustment": "adj_splitdiv",
    "period": "full",
    "overwrite": True,
    'userid': FRD_USER_ID
    }
download_stock_data(**params)

### Download All ticker ranges

In [ ]:
# Generate A-Z
EXT_ZIP_FILES_DIR = "/media/reggie/reg_ext/frd-historical/data/stock/zips/"
def download_stock_data_batch(ticker_start_letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ",
                              timeframes=['1min', '5min', '30min', '1hour', '1day'],
                              adjustments=['adj_split', 'adj_splitdiv', 'unadjusted'],
                              overwrite=True,
                              move=False,
                              move_dir=None,
):

    # Iterate through tickers and download data
    for ticker_first_letter in ticker_start_letters:
        print(f"Downloading data for {ticker_first_letter}")
        for adjustment in adjustments:
            for timeframe in timeframes:
                if timeframe != '1min' and adjustment == 'unadjusted':
                    continue
                
                # Create params.
                params = {
                    "asset_type": "stock",
                    "ticker_first_letter": ticker_first_letter,
                    "timeframe": timeframe,
                    "adjustment": adjustment,
                    "period": "full",
                    "overwrite": overwrite,
                    'userid': FRD_USER_ID
                }

                # Download.
                download_stock_data(**params)
                filename = f"{ticker_first_letter}_{adjustment}_{timeframe}.zip"
                download_path = os.path.join(ZIP_FILES_DIR, filename)
                print("Download path", download_path)
                
                fp_size = os.path.getsize(download_path)
                print(f"Downloaded {ticker_first_letter} {timeframe} {adjustment} data into file {download_path} with size {fp_size/(2**20)} MB")
                
                if move is True:
                    print(f"Zip file to move: {filename}")
                    move_path = os.path.join(move_dir, filename)
                    print(f"Moving {download_path} to {move_path}")
                    # move and overwrite if necessary
                    shutil.move(download_path, move_path)
    return 0
download_stock_data_batch(
    ticker_start_letters="QX",
    timeframes=['1min'],
    adjustments=['adj_splitdiv'],
    overwrite=True,
    move=True,
    move_dir=EXT_ZIP_FILES_DIR,
)

## Extract stock files from zips

In [ ]:
def extract_zip(src_path, dest_dir):
    """
    Extract a .zip file to a directory.
    Parameters
    ----------
    src_path : string
        Path to the .zip file to extract.
    
    Returns
    -------
    dest_dir : string
        Path to the directory where the .txt files were extracted.
    """
    #dest_dir = os.path.join(FRD_DATA_DIR, "tmp")
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    print(f"Extracting {src_path} to {dest_dir}")
    with zipfile.ZipFile(src_path, 'r') as zip_ref:
        zip_ref.extractall(path=dest_dir)
    n_files = len(os.listdir(dest_dir))
    print(f"Extracted {n_files} files from {src_path}")
    return dest_dir

def extract_stock_zips(**params):
    """Extract all .zip files for the given parameters."""
    # Unpack parameters
    adjustment = params['adjustment']
    timeframe = params['timeframe']
    ticker_first_letter = params['ticker_first_letter']
    
    ZIP_FILES_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/zips/"
    CSV_DEST_DIR = f"/media/reggie/reg_ext/frd-historical/data/stock/{timeframe}/csv/"

    # Find zip files
    print(f"Found {len(os.listdir(ZIP_FILES_DIR))} zip files in {ZIP_FILES_DIR}")
    zip_path = os.path.join(ZIP_FILES_DIR, f"{ticker_first_letter}_{adjustment}_{timeframe}.zip")
    print(CSV_DEST_DIR)
    if not os.path.isdir(CSV_DEST_DIR):
        os.makedirs(CSV_DEST_DIR)
        print(f"Created directory {CSV_DEST_DIR}")
    
    # Extract
    src_path = extract_zip(src_path=zip_path, dest_dir=CSV_DEST_DIR)
    print(src_path)

    files = [f for f in os.listdir(src_path) if f.endswith('.txt')]
    print(f"Number of files: {len(files)} in {src_path}")
    return 0

In [ ]:
#asset_type = "stock"
#ticker_first_letter = 'Q'
#timeframe = "1min"
#adjustment = "adj_splitdiv"
#period = "full"
#overwrite = True
#params = {
#    "asset_type": asset_type,
#    "ticker_first_letter": ticker_first_letter,
#    "timeframe": timeframe,
#    "adjustment": adjustment,
#    "period": period,
#    "overwrite": overwrite
#}
extract_stock_zips(**params)

In [ ]:
asset_type = "stock"
timeframe = "1min"
adjustment = "adj_splitdiv"
period = "full"
overwrite = True
#for ticker_first_letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
for ticker_first_letter in "BCDEFGHIJKLMNOPQRSTUVWXYZ":
    params = {
        "asset_type": asset_type,
        "ticker_first_letter": ticker_first_letter,
        "timeframe": timeframe,
        "adjustment": adjustment,
        "period": period,
        "overwrite": overwrite
    }
    extract_stock_zips(**params)

In [ ]:
from numpy import add


def get_csv_stats(file_path):
    """Get the number of rows and min, max number of columns of a csv file"""
    csv_stats = {'n_rows':0, 'min_columns':0, 'max_columns':0}
    columns_counts = []
    with open(file_path, 'r') as f:
        contents = f.readlines()
    
    for line in contents:
        columns_counts.append(len(line.split(',')))
    csv_stats['n_rows'] = len(contents)
    csv_stats['min_columns'] = min(columns_counts)
    csv_stats['max_columns'] = max(columns_counts)
    return csv_stats 

def get_directory_stats(directory, add_stats=True):
    """
    Get the file size of each file in a directory.
    Return it as a pandas dataframe.    
    """
    file_stats = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        file_size = os.path.getsize(file_path)
        file_stats.append({
            "file_path": file_path,
            "filename": os.path.basename(file_path),
            "file_size": file_size/(2**20)
        })
        if add_stats is True:
            # add columns on row and column stats
            csv_stats = get_csv_stats(file_path)
            file_stats[-1].update(csv_stats)

    file_stats_df = pd.DataFrame(file_stats)
    
    return file_stats_df
csv_stats_df = get_directory_stats(directory=CSV_DEST_DIR, add_stats=False)
csv_stats_df.sort_values("file_size", ascending=True, inplace=True)


In [ ]:
from tqdm import tqdm

# shuffle csv_stats_df
csv_stats_df = csv_stats_df.sample(frac=1)
n = len(csv_stats_df)
df_list = []

for i in tqdm(range(n)):
    row = csv_stats_df.iloc[i]
    file = row['file_path']
    filename = row['filename']
    #print(file)
    df = row.to_dict() 
    df.update(get_csv_stats(file))
    df = pd.DataFrame(df, index=[0])
    df_list.append(df)
df = pd.concat(df_list)
df.to_csv(os.path.join(FRD_DATA_DIR, "stock_csv_stats.csv"), index=False)
df.describe()

In [ ]:
df.query("max_columns != 6")

In [ ]:
import concurrent.futures
# List of all first letters to process
ticker_first_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

# Use ThreadPoolExecutor to execute tasks in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Create a list of futures
    futures = [
        executor.submit(
            extract_stock_zips(
                asset_type="stock",
                ticker_first_letter=ticker_first_letter,
                timeframe="1min",
                adjustment="adj_splitdiv",
                period="full",
                overwrite=True,)
        ) for ticker_first_letter in ticker_first_letters
    ]
    
    # Wait for all futures to complete
    for future in concurrent.futures.as_completed(futures):
        try:
            # Get the result of the future
            result = future.result()
        except Exception as e:
            print(f"An error occurred: {e}")
        else:
            # Process result (if necessary)
            print(f"Task completed with result: {result}")



In [ ]:

# combine all files in the list
columns = ["ticker", "date", "open", "high", "low", "close", "volume"]
data = [",".join(columns)+"\n"]
print(f"Writing to {csv_dest_path}")
with open(csv_dest_path, 'w') as f:
    f.writelines(data)

for i, file in enumerate(files):
    ticker = file.split('_')[0]
    file_path = os.path.join(src_path, file)
    print(f"Processing file {i+1}/{len(files)}: {file}")
    with open(file_path, 'r') as f:
        data = [ticker + ',' + line for line in f.readlines()]
        print(len(data))

        # append to csv
        print(f"Writing to {csv_dest_path}")
        with open(csv_dest_path, 'a') as f:
            f.writelines(data)

# convert dataframe to parquet
df = pd.read_csv(csv_dest_path, dtype={'ticker': str, 'date': str, 'open': float, 'high': float, 'low': float, 'close': float, 'volume': float})
print(df.shape)
parquet_fp = csv_dest_path.replace(".csv", ".parquet")
print(parquet_fp)
df.to_parquet(parquet_fp, index=False)

# clean up: remove individual .txt files
for file in files:
    os.remove(os.path.join(src_path, file))

## reorg directories

In [ ]:
def reorganize_data():
    base_path = "data/stock"
    adjustments = ['adj_split', 'adj_splitdiv', 'unadjusted']
    timeframes = ['1min', '5min', '30min', '1hour', '1day']
    
    for letter_dir in os.listdir(base_path):
        current_path = os.path.join(base_path, letter_dir)
        # Skip if not a directory
        if not os.path.isdir(current_path):
            continue
        
        for adj in adjustments:
            for timeframe in timeframes:
                # Define the source path for current adjustment and timeframe
                src_path = os.path.join(current_path, adj, timeframe)
                if not os.path.exists(src_path):
                    continue
                
                # Move each zip file into a new structure based on its name
                for file in os.listdir(src_path):
                    ticker_symbol = file.split('_')[0]  # Assuming filename format is "Symbol_adj_timeframe.zip"
                    new_path = os.path.join(base_path, ticker_symbol, adj, timeframe)
                    if not os.path.exists(new_path):
                        os.makedirs(new_path)
                    
                    # Move file from old structure to new structure
                    shutil.move(os.path.join(src_path, file), os.path.join(new_path, file))
                
                # Cleanup: remove the now-empty directories
                if not os.listdir(src_path):
                    os.rmdir(src_path)
            # Check and remove empty adjustment directory
            adj_path = os.path.join(current_path, adj)
            if not os.listdir(adj_path):
                os.rmdir(adj_path)
                
        # Check and remove empty letter directory
        if not os.listdir(current_path):
            os.rmdir(current_path)

#if __name__ == "__main__":
#    reorganize_data()

# Load data into database tables

In [ ]:
# stand up database for extracted txt files
# create table for each ticker

# Example

In [ ]:
ticker = "RDFN"
filename = [f for f in files if f.find(ticker) != -1][0]
fp = os.path.join(ZIP_SRC_FP, filename)
print(fp)
#prices_df = pd.read_csv(fp, sep=",")
prices_df = pd.read_csv(fp, sep=",", names=['date', 'open', 'high', 'low', 'close', 'volume'])
print(prices_df.shape)
prices_df['date'] = prices_df['date'].apply(pd.to_datetime)

prices_df['ticker'] = ticker
prices_df['day'] = prices_df['date'].apply(lambda d: d.day)
prices_df['month'] = prices_df['date'].apply(lambda d: d.month)
prices_df['hour'] = prices_df['date'].apply(lambda d: d.hour)
prices_df['minute'] = prices_df['date'].apply(lambda d: d.minute)
prices_df['period'] = prices_df['month'].apply(lambda x: str(x).zfill(2)) + \
    prices_df['day'].apply(lambda x: str(x).zfill(2)) + \
    prices_df['hour'].apply(lambda x: str(x).zfill(2)) + \
    prices_df['minute'].apply(lambda x: str(x).zfill(2))
prices_df['period'] = prices_df['period'].astype(int)
prices_df.describe()
prices_df

In [ ]:
plt.figure(figsize=(16,8))
month, day = 10, 23
for day in range(1, 24):
    X = prices_df.query(f"month == {month}").query(f"day == {day}")
    if len(X) > 0:
        X['open_scaled'] = X['open']/X['open'].values[0]
        plt.plot(X['period'], X['open_scaled'])
plt.show()

# Overnight returns

In [ ]:
import numpy as np

In [ ]:
def is_open(dt, buffer=0):
    dt = pd.to_datetime(dt)
    return np.all((dt.hour == 9, dt.minute == 30 + buffer))

def is_close(dt, buffer=0):
    dt = pd.to_datetime(dt)
    return np.all((dt.hour == 16, dt.minute == 0 - buffer))
prices_df['at_open'] = prices_df['date'].apply(lambda x: is_open(x,1))
prices_df['at_close'] = prices_df['date'].apply(lambda x: is_close(x,1))

In [ ]:
overnight_df = prices_df.query("(at_close == True) | (at_open == True)").sort_values('date')
overnight_df['open_lag1'] = overnight_df['open'].shift(1)
overnight_df['close_lag1'] = overnight_df['close'].shift(1)
overnight_df['delta_price'] = overnight_df['open'] - overnight_df['close_lag1']
overnight_df['lret'] = np.log(overnight_df['open']/overnight_df['close_lag1'])
overnight_df[['date','period', 'close_lag1', 'open', 'delta_price', 'lret']]

In [ ]:
plt.figure(figsize=(16,8))
X = overnight_df[['date', 'lret']].reset_index()
X = X.assign(index=list(range(len(X))))
plt.bar(x=X['index'], height=X['lret'])
plt.show()